# AUTOMATIC DIFFERENTIATION WITH ```torch.autograd```
- **Back propagation** 은 인공신경망을 학습할 때 가장 많이 사용되는 알고리즘임
- 역전파에서 모델의 파라미터는 loss function 에 대한 **gradient** 에 따라 조정됨

- 파이토치는 ```torch.autograd``` 라는 내장 미분 엔진을 이용하여 gradient 를 계산함

- 1층으로 구성된 간단한 인공신경망을 예로 들겠음
- 입력 ```x```, 파라미터 ```w``` 와 ```b```, loss function 을 정의해야 함

In [1]:
import torch

x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad = True)
b = torch.randn(3, requires_grad = True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

---
# 1. Tensors, Functions and Computational graph
- 위 모델에서 최적화할 파라미터는 ```w, b``` 임
- 그러기 위해선 loss function 에 대한 각 변수의 gradient 를 계산해야 함
- 이를 위해 파라미터 텐서의 ```requires_grad``` 특성을 설정함

>변수의 ```requires_grad``` 는 ```x.requires_grad_(True)``` 형식으로도 설정 가능

- backward propagation 에 참조될 함수 객체는 텐서의 ```grad_fn``` 특성에 저장돼 있음

In [2]:
print("Gradient function for z = ", z.grad_fn)
print("Gradient function for loss = ", loss.grad_fn)

Gradient function for z =  <AddBackward0 object at 0x00000278FB8B8BE0>
Gradient function for loss =  <BinaryCrossEntropyWithLogitsBackward object at 0x00000278FB8B8B20>


---
# 2. Compution Gradients
- weight 파라미터를 최적화하기 위해선 각 파라미터로 loss function 을 편미분하여 derivative 를 구해야 함
- ```loss.backward()``` 를 호출하여 이 derivative 를 구할 수 있음
- 호출한 뒤 ```w.grad, b.grad``` 에서 해당 정보를 확인 가능함

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0045, 0.3152, 0.2234],
        [0.0045, 0.3152, 0.2234],
        [0.0045, 0.3152, 0.2234],
        [0.0045, 0.3152, 0.2234],
        [0.0045, 0.3152, 0.2234]])
tensor([0.0045, 0.3152, 0.2234])


>- 이때 ```grad``` 특성을 얻을 수 있는 노드는 ```requires_grad=True``` 로 설정된 노드들 뿐임
>- 주어진 그래프에 대해 ```backward```는 **오직 한 번만 호출 가능**
>- 같은 그래프에 대해 여러 번 호출이 필요한 경우엔 ```backward``` 호출 시 ```retain_graph=True``` 설정해야 함 

---
# 3. Disabling Gradient Tacking
- ```requires_grad=True``` 로 설정된 텐서들은 기본적으로 자신의 연산 기록을 계속 추적함
- 그러나 가끔은 그럴 필요가 없음 (그냥 입력 데이터를 처리해볼 경우)
- ```torch.no_grad()``` 블록을 이용하여 연산 기록의 추적을 멈출 수 있음

In [5]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


- 다른 방법으로는 텐서의 ```datach()``` 메서드를 사용하는 것

In [6]:
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


> **gradient tracking** 을 중단하는 몇몇 이유
>- 모델의 파라미터를 **frozen parameters** 로 만들기 위해 (주로 사전학습된 네트워크를 파인튜닝할 때)
>- forward 연산을 할 때 **연산 속도를 높이기 위해**

---
# 4. More on Computational Graphs
- 콘셉상으로 autograd 는 **DAG(directed acyclic graph)** 내에서 만들어진 데이터(텐서)와 실행된 모든 연산을 기록함
- DAG 는 **Function** 객체로 구성됨
- DAG 에서 입력 텐서는 leaves, 출력 텐서는 roots 가 됨
- roots 에서부터 leaves 까지 추적하여 **chain rule** 에 따라 gradient 를 자동으로 계산함

>**forward 에서 autograd 가 동시에 수행하는 두 가지 작업**
>- 결과 텐서를 계산하기 위해 필요한 연산
>- 각 연산의 **gradient function** 을 DAG 에서 유지

>**DAG 에서 ```.backward()``` 가 호출될 때 autograd 가 수행하는 작업**
>- 각 ```.grad_fn```에 대해 gradient 계산
>- 각 텐서의 ```.grad``` 특성에 gradinet 를 축적
>- chaing rule 에 따라 leaf 에 해당하는 텐서들에 모두 전파

>- ```.backward()``` 를 아무런 파라미터 없이 호출하는 것은 ```.backward(torch.tensor(1.0))``` 과 동일함
>- 이는 **scalar-valued function*** 의 그레디언트를 구할 때 유용한 방법이라고 함 (즉 모델의 손실 함수)
>- **만약 그레디언트를 구할 대상이 스칼라가 아니라면 별도의 파라미터 필요**